## 01 Exploratory Data Analysis

### Environment

Designed for Python 3.11.9 on Windows 10

##### Install Dependencies

In [ ]:
# Install from requirements.txt
!python -m pip install -r requirements.txt

In [ ]:
# Alternative for running in Colab etc
!pip install --upgrade pip
!pip install numpy==1.24.1
!pip install pmdarima==2.0.4
!pip install ibis-framework[duckdb]==9.5.0
!pip install pandas==2.2.3
!pip install matplotlib==3.9.2
!pip install skforecast==0.14.0

### Load and Preprocess Data

Key tools:
- [Ibis](https://ibis-project.org/) on a [DuckDB](https://duckdb.org/) backend with transformations to [Pandas](https://pandas.pydata.org/docs/index.html) for certain operations.
- Some excessively-verbose functions spun out to a data prep module 

In [ ]:
import numpy as np
import pandas as pd
import ibis
import matplotlib.pyplot as plt
import skforecast
import pmdarima
import methods.prep as prep
import methods.vis as vis
import importlib

In [ ]:
importlib.reload(vis)
importlib.reload(prep)

##### Load Individual Datasets

In [ ]:
con = ibis.connect("duckdb://")
data_path = 'data_eda/'

In [ ]:
# Load FRED datasets
fred_dataset_names = [
    'consumer-sentiment',
    'cpi',
    'hourly-wage',
    'house-median-price',
    'house-starts',
    'unemployment',
]

fred_datasets = {}

print('\nFRED Datasets (single-variable time series)')
for name in fred_dataset_names:
    fred_datasets[name] = ibis.read_csv(
        data_path + name + '.csv',
        dateformat='%m/%d/%Y')
    print(name + ': ' + str(fred_datasets[name].to_pandas().shape))

In [ ]:
# Load investing.com datasets

inv_dataset_names = [
    'copper', 
    'corn', 
    'gold',
    'lumber',
    'natural-gas',
    'oil-wti',
    'r2000', 
    'soy',
    'sp500',
    'vix-volatility'
]

traded_commodities = [
    'copper'
]

inv_datasets = {}

print('Investing.com Datasets (standard prices and volume time series)')
for name in inv_dataset_names:
    inv_datasets[name] = ibis.read_csv(
        data_path + name + '.csv',
        dateformat='%m/%d/%Y')
    print(name + ': ' + str(inv_datasets[name].to_pandas().shape))


    

##### Initial Preprocessing - Individual Datasets

In [ ]:
for key in inv_datasets.keys():
    #Drop unnecessary columns
    inv_datasets[key] = inv_datasets[key].drop('Vol.','Change %')

    #Add trading day flag to traded columns
    if key in traded_commodities:
        inv_datasets[key] = inv_datasets[key].mutate(
            TRADING_DAY = True
        )

    #Convert column names to ALL CAPS
    inv_datasets[key] = inv_datasets[key].rename('ALL_CAPS')

    # Convert PRICE, OPEN, HIGH, and LOW to float64s if they are strings
    if inv_datasets[key]['PRICE'].type().is_string():
        inv_datasets[key] = inv_datasets[key].mutate(
            PRICE = inv_datasets[key]['PRICE'].replace(',','').cast('float64'))    

    if inv_datasets[key]['OPEN'].type().is_string():
        inv_datasets[key] = inv_datasets[key].mutate(
            OPEN = inv_datasets[key]['OPEN'].replace(',','').cast('float64'))   

    if inv_datasets[key]['HIGH'].type().is_string():
        inv_datasets[key] = inv_datasets[key].mutate(
            HIGH = inv_datasets[key]['HIGH'].replace(',','').cast('float64'))  

    if inv_datasets[key]['LOW'].type().is_string():
        inv_datasets[key] = inv_datasets[key].mutate(
            LOW = inv_datasets[key]['LOW'].replace(',','').cast('float64'))   

    # Add prefixes to each column based on the name of the dataset
    names_map = {
        f"{key}_{col}" : col 
            for col in inv_datasets[key].columns
            if col != 'DATE'}
    inv_datasets[key] = inv_datasets[key].rename(names_map)

for k,v in inv_datasets.items():
    print(v.head())

In [ ]:
# Fix column names in FRED data and drop unused rows
for key in fred_datasets.keys():
    #Rename second column (data) to name of dataset
    old_col_name = fred_datasets[key].columns[1]
    new_col_name = key
    fred_datasets[key] = fred_datasets[key].rename({new_col_name: old_col_name})

    #Convert timestamps to dates
    fred_datasets[key] = fred_datasets[key].mutate(
        DATE = fred_datasets[key]['DATE'].cast('date'))

    #Convert to snake-case
    fred_datasets[key] = fred_datasets[key].rename('ALL_CAPS')

    #Drop unused rows - but keep some in 2006 to support forward fill
    fred_datasets[key] = fred_datasets[key].filter(fred_datasets[key].DATE.year() > 2005)

# Convert string to float64 in consumer sentiment data   
fred_datasets['consumer-sentiment'] = fred_datasets['consumer-sentiment'].mutate(
    CONSUMER_SENTIMENT  = fred_datasets['consumer-sentiment']['CONSUMER_SENTIMENT'].replace(',','').cast('float64'))

for k,v in fred_datasets.items():
    #print(k)
    print(v.head())

##### Merge Datasets & Forward Fill Missing Data

While the datasets all cover the desired 2007-2024 date range, the specific dates covered by each dataset vary widely. 

In [ ]:
# # Merge all tables from the Investing.com dataset
# inv_data = prep.merge_tables(
#     inv_datasets, 
#     join_key = 'DATE',
#     join_type = 'outer')

# # Merge all tables from the FRED dataset together
# fred_data = prep.merge_tables(
#     fred_datasets, 
#     join_key = 'DATE', 
#     join_type = 'outer')

# # Bring dates from inv table into FRED so values can be imputed
# fred_data = prep.merge_tables(
#     tables_to_merge={
#         'fred' : fred_data,
#         'inv' : inv_data.select('DATE')
#     },
#     join_key = 'DATE',
#     join_type = 'outer'
# )

# # Run the forward-fill imputation - fred data
# fred_data = prep.impute_forward_fill(
#     data = fred_data,
#     sort_by = 'DATE')

# # Filter out early nulls from FRED
# fred_data = fred_data.filter(fred_data.DATE.year() > 2006)

# # Run the forward-fill imputation - investing.com data
# # TODO decide whether to keep this
# inv_data = prep.impute_forward_fill(
#     data = inv_data,
#     sort_by = 'DATE')

# # Finally, merge the investing.com and FRED data
# # Using an inner join to exclude any days which aren't trading days
# data = prep.merge_tables(
#     {
#         'inv' : inv_data, 
#         'fred' : fred_data,
#     },
#     join_key = 'DATE',
#     join_type='inner')

# # TODO For TRADING_DAY columns,  fill in NaNs with FALSE 


In [ ]:
import datetime
# Merge all tables from the Investing.com dataset
inv_data = prep.merge_tables(
    inv_datasets, 
    join_key = 'DATE',
    join_type = 'outer')

# Merge all tables from the FRED dataset together
fred_data = prep.merge_tables(
    fred_datasets, 
    join_key = 'DATE', 
    join_type = 'outer')

# Filter out early nulls from FRED
fred_data = fred_data.filter(fred_data.DATE.year() > 2006)

# Create a table with all of the dates in range
daterange = []
current_date = datetime.date(2006,1,1)
while current_date <= datetime.date(2024, 10, 31):
    daterange.append(current_date)
    current_date = current_date + datetime.timedelta(days=1)
daterange = pd.DataFrame(daterange, columns=['DATE'])

# Finally, merge the investing.com, FRED data, and daterange
# using an outer join to ensure there is a row for every date
data = prep.merge_tables(
    {
        'inv' : inv_data, 
        'fred' : fred_data,
        'dates': daterange
    },
    join_key = 'DATE',
    join_type='outer')

# Forward-fill impute all numeric data 
data = prep.impute_forward_fill_numerics(
    data = data,
    sort_by = 'DATE')

In [ ]:
df = data.filter(data.DATE.year() >= 2007).filter(data.DATE.year() <= 2019).to_pandas()

In [ ]:
data.to_pandas().shape

In [ ]:
df.isna().sum()

##### Data Comparability

Comparisons over time can be improved by calculating real (inflation-corrected) prices and adjusting housing starts to a per-capita rate.  This will also take some of the trends out of the data, improving stationarity for forecasting purposes.

In [ ]:
# TODO Add REAL price columns by controlling for time-lagged CPI 
# (divide all prices by relative CPI, indexed to base year)
# Applies to all commodities and equities indexes (ex VIX)
# as well as hourly wages and median house prices

In [ ]:
# TODO Add a US_Housing_Starts_Per_Capita column 
# TODO would need to bring in population indicator
# to provide a clearer economic health indicator

##### Data Availability Corrections

Data that is not yet available at prediction time can't be used to make predictions.  Many macroeconomic indicators are unavailable for weeks or months after the dates they pertain to as the underlying data must be reported and processed.  At this point, we'll avoid this concern by lagging all macroeconomic indicators by 90 days.

In [ ]:
# TODO Offset macroeconomic indicators to align with when they're reported/available 
# or simplify by lagging all of them 3 months?

##### Feature Engineering for Forecasting

In [ ]:
# TODO Add LAG columns - ~3 month, ~1 year, ~3 years?

In [ ]:
# TODO Add XMA 10-day, 30-day, and 90-day columns to help with technical analysis?

### Dev-Holdout Data Split 

Starting each period with a crisis!

In [ ]:
training_end_year = 2019

# Split the data into model development (2007-01-01 to 2019-12-31) 
dev_data = data.filter(data.DATE.year() >= 2007).filter(data.DATE.year() <= training_end_year)

# and final holdout (2020-01-01 to 2024-10-31)
holdout_data = data.filter(data.DATE.year() > training_end_year)

### Feature Correlations

Not especially useful since this is time-series data, but a quick look doesn't hurt.  Mostly 

In [ ]:
# TODO switch these to look at only economic indicators and 
# REAL prices, wages, and only closing prices
dev_data.to_pandas().select_dtypes(include=np.number).corr()

**Copper vs other commodities** - we see a mix of very tight positive correlations, in some cases almost perfectly linear.  Some positive correlation is to be expected since we haven't controlled for inflation yet, but inflation wasn't all that high during the dev period.  However, we also see surprisingly loose correlations in some cases, including separate price spikes.  There's some curious multimodality in both price distributions and price correlations.

In [ ]:
# TODO switch these to REAL prices
vis.plot_feature_correlation_matrix(
    dev_data.select(
        'copper_PRICE',
        'oil-wti_PRICE',
        'lumber_PRICE',
        'corn_PRICE',
        'natural-gas_PRICE',
        'soy_PRICE',
    ).to_pandas())

**Copper vs economic indicators** - Also shows a mix of straightfoward linear correlation and curious drift and multimodality.  While some of the linear correlation can be attributed to inflation, much of it cannot be over this time frame.

In [ ]:
# TODO Switch to REAL prices
vis.plot_feature_correlation_matrix(
    dev_data.select(
        'copper_PRICE',
        'CONSUMER_SENTIMENT',
        'r2000_PRICE',
        'UNEMPLOYMENT',
        'HOUSE_STARTS'
    ).to_pandas())

### Linear Plots & Preliminary Stationarity Analysis

For forecasting, features should be stationary, meaning that no significant trends or seasonal patterns should be present in the data.  The mean and variance should be consistent throughout the time period.

No features are stationary without differencing.  The trends are enormous, and while seasonality is difficult to detect at this level, it's almost certainly present.  Many of these look pretty good at first differencing, but some definitely need to be differenced at least one more time.

In [ ]:

column_list = [
    'DATE',
    'copper_PRICE',
    'CONSUMER_SENTIMENT',
    'r2000_PRICE',
    'UNEMPLOYMENT',
    'HOUSE_STARTS'
]
df = dev_data.select(column_list).to_pandas()
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values('DATE')
fig, axes = vis.plot_time_series_diffs(df)

##### Common Stationarity Tests

In [ ]:
# TODO add functional stationarity tests & interpretation
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
adfuller(df['copper_PRICE'])

In [ ]:
stationarity_tests = {
    'adfuller' :  adfuller(df['copper_Price']),
    'kpss' : kpss(df['copper_Price']) 
}

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

### Baseline Models

Key tools:
- [skforecast](https://skforecast.org/)
- [pmdarima](https://github.com/alkaline-ml/pmdarima)
- [sklearn scaling]()

In [ ]:
from skforecast.sarimax import Sarimax
from sklearn.preprocessing import StandardScaler

##### Remove Nontrading Days

At this point, all required lags and moving-average values have been calculated.  For calculating deltas, we will only want trading days. 

##### Scaling

##### Pure ARIMA Model

In [ ]:
df = dev_data.to_pandas()
df['DATE'] = pd.to_datetime(df['DATE'])
pdq = (1,1,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(
    y = df['copper_PRICE'])
model.summary()

##### Simple ARIMAX Models

Using opening prices and volatility only to predict copper closing price

In [ ]:
df = dev_data.to_pandas()
exog_cols = [col for col in df.columns if '_OPEN' in col]
exog = df[exog_cols]
pdq = (1,1,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(
    y = df['copper_PRICE'],
    exog = exog)
model.summary()

In [ ]:
df = dev_data.to_pandas()
(df['copper_OPEN'] - df['copper_OPEN'].mean())/df['copper_OPEN'].std()

In [ ]:
df = dev_data.to_pandas()
exog_cols = [col for col in df.columns if '_OPEN' in col]
exog = df[exog_cols]
exog = exog.drop(['natural-gas_OPEN','gold_OPEN','corn_OPEN'], axis='columns')
exog = (exog - exog.mean())/exog.std()

target = df['copper_PRICE']
target = (target - target.mean())/target.std()

pdq = (1,1,1) # p autoregression lags, d differences, q moving average
model = Sarimax(order = pdq)
model.fit(
    y = target,
    exog = exog)
model.summary()